In [32]:
import os
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
def get_RNCtoUD_table(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        tags_table = f.read()
    tags = tags_table.split('\n')
    tags_dict = {}
    for tag in tags:
        if tag != '':
            parts = tag.split('\t')
            tags_dict[parts[0]] = parts[1]
    return tags_dict

In [3]:
def get_list_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
    some_list = text.split('\n')
    return some_list

In [5]:
def get_sents(text):
    body_text = re.search('<body>(.+)</body>', text, re.DOTALL)
    text = body_text.group(1).strip()
    sents = text.split('<se>')
    return sents

### Все, что связано со словами

In [6]:
# Получает все части разбора из тега <ana>

def get_word_info(word):
    soup = BeautifulSoup(word, 'html.parser')
    lemma = str(soup.text)
    ana = soup.find('ana')
    lex = str(ana.get('lex'))
    joined = str(ana.get('joined'))
    gram = str(ana.get('gr'))
    # unify gram
    gram = gram.replace('=', ',')
    gram = gram.replace('-', '')
    return lemma, lex, gram, joined

In [7]:
# Решает, что делать с неоднозначными соответствиями из tags_dict

def decide_pos(pos, grammems, lex, cconj_list):
    propn_grams = ['persn', 'famn', 'patrn', 'zoon', 'topon']
    if pos == 'V':
        if lex == 'быть':
            answer = 'AUX'
        else:
            answer = 'VERB'
    
    elif pos == 'CONJ':
        if lex.lower() in cconj_list:
            answer = 'CCONJ'
        else:
            answer = 'SCONJ'
    
    else:
        for g in propn_grams:
            if g in grammems:
                answer = 'PROPN'
                break
            else:
                answer = 'NOUN'
        if 'abbr' in grammems and lex[0].isupper():
            answer = 'PROPN'
    return answer

In [8]:
# Меняет помету части речи

def change_pos(pos, tags_dict, grammems, lex, cconj_list):
    double = ['V', 'CONJ', 'S']
    ud_pos = ''
    if pos in double:
        ud_pos =  decide_pos(pos, grammems, lex, cconj_list)
    else:
        ud_pos = tags_dict[pos]
    return ud_pos

In [9]:
def change_grammems(grammems, pos, tags_dict):
    ud_list = []
    for g in grammems:
        ud_list.append(tags_dict[g])
    if pos == 'INIT' and 'Abbr=Yes' not in ud_list:
        ud_list.append('Abbr=Yes')
    ud_gram = '|'.join(sorted(ud_list))
    return ud_gram

In [10]:
# Собирает линию с одним словом в новом формате

def make_word_line(word, k, cconj_list, tags_dict):
    word_ana = word.split('</w>')[0]
    lemma, lex, gram, joined = get_word_info(word_ana)
    if ',' in gram:
        pos = gram.split(',')[0]
        grammems = gram.split(',')[1:]
        ud_gram = change_grammems(grammems, pos, tags_dict)
    else:
        pos = gram
        grammems = ''
        ud_gram = '_'
    if joined != 'None':
        ud_gram = ud_gram + '|' + joined
    ud_pos = change_pos(pos, tags_dict, grammems, lex, cconj_list)
    line = str(k) + '\t' + lemma + '\t' + lex + '\t' + ud_pos + '\t_\t' + ud_gram + '\t_'*4
    return line

### Все, что связано с пунктуацией и тегами

In [11]:
# делит теги на открывающие и закрывающие

def divide_tags(tags):
    last_tags = re.search(r'.*(</?\w.*>\n*)*</\w*>', tags, re.DOTALL)
    
    if last_tags:
        next_tags = tags.replace(last_tags.group(), '')
        return last_tags.group(), next_tags
    else: 
        next_tags = tags
        return '', next_tags

In [12]:
def find_punct_tags(after):
    tags = re.findall('<[^>]*>', after, re.DOTALL)
    
    punct = after
    for t in tags:
        punct = punct.replace(t, '').strip()
    if tags:
        tags = ''.join(tags)
        tags = tags.replace('</se>', '')
        current_tags, next_tags = divide_tags(tags)
        return punct, current_tags, next_tags
    else:
        punct = after.strip()
        return punct, '', ''

In [14]:
def make_punct_line(punct, k, sym_list):
    punct = punct.replace('\n', '').replace('\t', '').replace(' ', '')
    lines = []
    count = 0
    for i in range(len(punct)):
        if punct[i] in sym_list:
            line = str(k) + '\t' + punct[i] + '\t' + punct[i] + '\t' + 'SYM' + '\t_'*6 
            lines.append(line)
            k += 1
            count += 1
        else:
            if count != 0 and len(punct) > 1 and punct[i-1] == punct[i]:
                parts = lines[count-1].split('\t')
                parts[1] = parts[1] + punct[i] 
                parts[2] = parts[2] + punct[i]
                lines[count-1] = '\t'.join(parts)
            else:
                line = str(k) + '\t' + punct[i] + '\t' + punct[i] + '\t' + 'PUNCT' + '\t_'*6
                lines.append(line)
                k += 1        
                count += 1
    return '\n'.join(lines), k

### Главное: трансформирует предложения

In [15]:
def add_opening_punct(words, k, sym_list, next_tags, new_sent):
    punct, more_last_tags, more_next_tags = find_punct_tags(words[0])
    if punct != '':
        punct_line, k = make_punct_line(punct, k, sym_list)
        if next_tags != '':
            p_lines = punct_line.split('\n')
            new_p_line = p_lines[0][:-1] + 'before:' + next_tags.strip() 
            p_lines[0] = new_p_line
            punct_line = '\n'.join(p_lines)
            next_tags = ''
        new_sent.append(punct_line.strip())
    return new_sent, k, next_tags, more_next_tags

In [16]:
def add_last_tags(line, last_tags):
    line = line.strip()
    if line[-1] == '_':
        line = line[:-1]  + last_tags.replace('</se>', '').strip()
    else:
        line = line  + '|' + last_tags.replace('</se>', '').strip()
    last_tags = ''
    return line, last_tags

In [17]:
def add_next_tags(line, next_tags, more_tags):
    line = line.strip()
    if line[-1] == '_':
        line = line[:-1]  + 'before:' + next_tags.replace('</se>', '').strip() + more_tags.replace('</se>', '').strip()
    else:
        line = line  + '|' + 'before:' + next_tags.replace('</se>', '').strip() + more_tags.replace('</se>', '').strip()
    next_tags = ''
    return line, next_tags

In [18]:
def transform_sents(sents, sym_list, cconj_list, tags_dict):
    new_sents = []
    next_tags = sents[0]

    for sent in sents[1:]:
        new_sent = []
        sent = sent.strip()
        words = sent.split('<w>')
        k = 1
        punct, more_last_tags, more_next_tags = find_punct_tags(words[0])
        if punct != '':
            new_sent, k, next_tags, more_next_tags = add_opening_punct(words, k, sym_list, next_tags, new_sent)

        # between words
        for word in words[1:-1]:
            word_line = make_word_line(word, k, cconj_list, tags_dict)
            if next_tags != '':
                word_line, next_tags = add_next_tags(word_line, next_tags, more_next_tags)
            after_word = word.split('</w>')[1]
            punct, last_tags, next_tags = find_punct_tags(after_word)
            if last_tags != '':
                word_line, last_tags = add_last_tags(word_line, last_tags)
            new_sent.append(word_line)
            k += 1
            if punct != '':
                punct_line, k = make_punct_line(punct, k, sym_list)
                new_sent.append(punct_line.strip())

        # between sents (last word) 
        after_sent = words[-1].split('</w>')[1]
        word_line = make_word_line(words[-1], k, cconj_list, tags_dict)
        k += 1
        # add next_tags
        if next_tags != '':
            word_line, next_tags = add_next_tags(word_line, next_tags, '')

        punct, last_tags, next_tags = find_punct_tags(after_sent)
        # add last_tags
        if punct != '':
            punct_line, k = make_punct_line(punct, k, sym_list)
            if last_tags != '':
                punct_line, last_tags = add_last_tags(punct_line, last_tags)
        else:
            if last_tags != '':
                word_line, last_tags = add_last_tags(word_line, last_tags)
        new_sent.append(word_line)
        if punct != '':
            new_sent.append(punct_line.strip())
        new_sents.append('\n'.join(new_sent))
    return new_sents

### Составляет предложение без разбора

In [19]:
def next_is_punct(next_part):
    return next_part != '' and next_part.split('\t')[3] == 'PUNCT'


def make_sentence_from_parsed(parsed_sentence, cov):
    result = ''
    lines = parsed_sentence.split('\n')
#     cov = 0
    for i, line in enumerate(lines):
        space_after = False
       
        if line == '':
            continue
        parse_parts = line.split('\t')
        word = parse_parts[1]
        
        current_part = word
        with_space = ['(', '<', '{', '[', '«', '--']
        if word == '"':
            cov += 1
        
        if word in with_space or word == '"' and cov % 2 == 1:
            if word == '--':
                current_part += ' '
                space_after = True
        elif i < len(lines) - 1:
            if next_is_punct(lines[i + 1]):
                next_word = lines[i + 1].split('\t')[1]
                if next_word in with_space or next_word == '"' and cov % 2 == 0:
                    current_part += ' '
                    space_after = True
            else:
                if word != '-':
                    current_part += ' '
                    space_after = True
        result += current_part
        
        if i == len(lines) - 1:
            space_after = True
        if not space_after:
            parts = lines[i].split('\t')
            if parts[9] == '_':
                parts[9] = 'SpaceAfter=No'
            else:
                parts[9] = 'SpaceAfter=No' + '|' + parts[9]
            lines[i] = '\t'.join(parts)
        parsed_sent = '\n'.join(lines)
        
    result = result.replace('`', '')
    result = result.replace('ё', 'е')
    result = result.replace('Ё', 'Е')
    return result, parsed_sent, cov

### Дополнительные изменения
* добавление пометы joined= в 10 столбик
* добавление пометы wf=  в 10 столбик

In [20]:
def move_joined(parts, joined_type):
    add_line = '|joined="'+ joined_type[1:] + '"'
    if joined_type in parts[5]:
        parts[5] = parts[5].replace(joined_type, '')
        if parts[9] == '_':
            parts[9] = add_line[1:]
        else:
            parts[9] = parts[9]+ '|' + add_line[1:]
    return parts

In [21]:
def move_word_form(parts, sym_type, replace_sym):
    if sym_type in parts[1]:
        if parts[9] == '_':
            parts[9] = 'wf=' + parts[1]
        else:
            parts[9] = parts[9]+ '|' + 'wf=' + parts[1]
        parts[1] = parts[1].replace(sym_type, replace_sym)
    return parts

### Присоединяет одиночные точки к предыдущему токену

In [22]:
def change_numbers(lines):
    for j, line in enumerate(lines):
        parts = line.split('\t')
        parts[0] = str(j + 1)
        lines[j] = '\t'.join(parts)
    return lines

def modify_space_after(parts):
    spaces = parts[9].split('|')
    for k in range(len(spaces)):
        if spaces[k].startswith('SpaceAfter'):
            spaces.pop(k)
            break
    for k in range(len(spaces)):
        if spaces[k].startswith('wf='):
            spaces[k] = spaces[k] + '.'
            break
    return spaces

def deal_with_dots(sent):
    pop_list = []
    lines = sent.split('\n')
    for i in range(len(lines[1:])):
        if lines[i].split('\t')[1] == '.':
            flag = True
            for j in range(i, len(lines)):
                if lines[j].split('\t')[3] != 'PUNCT' and lines[j].split('\t')[3] != 'SYM':
                    flag = False
            if not flag and i > 0:
                parts = lines[i-1].split('\t')
                parts[1] = parts[1] + '.'
                if parts[9] == '_':
                    parts[9] = lines[i].split('\t')[9]
                else:
                    spaces = modify_space_after(parts)
                    parts[9] =  '|'.join(spaces)
                    if lines[i].split('\t')[9] != '_':
                        parts[9] = lines[i].split('\t')[9] + '|' + parts[9]
                    elif parts[9] == '':
                        parts[9] = '_'
                lines[i-1] = '\t'.join(parts)
                pop_list.append(i)
                
    for i in pop_list[::-1]:
        lines.pop(i)
    lines = change_numbers(lines)
    result = '\n'.join(lines)
    return result

In [23]:
def complete_transformation(new_sents):
    new_text = ''
    cov = 0
    for k, sent in enumerate(new_sents):
        only_words_sent, space_sent, cov = make_sentence_from_parsed(sent, cov)
        lines = space_sent.split('\n')
        for i in range(len(lines)):
            parts = lines[i].split('\t')
            if '</p>' in parts[9]:
                if parts[9] == '_':
                    parts[9] = 'SpaceAfter=\\n'
                elif 'SpaceAfter' in parts[9]:
                    parts[9] = parts[9].replace('SpaceAfter=No', 'SpaceAfter=\\n')
                else:
                    parts[9] = 'SpaceAfter=\\n' + '|' + parts[9]
            parts = move_joined(parts, '|hyphen')
            parts = move_joined(parts, '|together')
            parts = move_word_form(parts, '`', '')
            parts = move_word_form(parts, 'ё' , 'е')
            parts = move_word_form(parts, 'Ё', 'Е')
            lines[i] = '\t'.join(parts)
        new_sents[k] == '\n'.join(lines)
        space_sent = '\n'.join(lines)
        space_sent = deal_with_dots(space_sent)

        if k != 0:
            if '</p>' in new_sents[k-1].split('\n')[-1]:
                new_text = new_text + '# newpar\n# sent_id = ' + str(k+1) + '\n# text = ' + only_words_sent + '\n' + space_sent + '\n\n'
            else:
                new_text = new_text + '# sent_id = ' + str(k+1) + '\n# text = ' + only_words_sent + '\n' + space_sent + '\n\n'
        else:
            new_text = new_text + '# newdoc\n# sent_id = ' + str(k+1) + '\n# text = ' + only_words_sent + '\n' + space_sent + '\n\n'
    return new_text

In [54]:
def RNCtoUD_one_text(filepath, sym_list, cconj_list, tags_dict):
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()
    sents = get_sents(text)
    new_sents = transform_sents(sents, sym_list, cconj_list, tags_dict)
    new_text = complete_transformation(new_sents)
    conllu_name = '.'.join(filepath.split('.')[:-1]) + '.conllu' 
    os.makedirs(os.path.dirname('RNCtoUD_result\\' + conllu_name), exist_ok=True)
    with open('RNCtoUD_result\\' + conllu_name, 'w', encoding='utf-8') as f:
        f.write(new_text)

In [55]:
def main(root_dir):
    tags_dict = get_RNCtoUD_table('RNCtoUDtable-ext.txt')
    cconj_list = get_list_from_file('data_lists/CCONJ_list.txt')
    sym_list = get_list_from_file('data_lists/symbols.txt')
    walk = [(x, y, z) for x, y, z in os.walk(root_dir)]
    for root, dirs, files in tqdm(walk):
        for f in files:
            f_path = os.path.join(root, f)
            RNCtoUD_one_text(f_path, sym_list, cconj_list, tags_dict)

In [56]:
main('test_folder')

100%|████████████████████████████████████████████| 1/1 [00:08<00:00,  8.26s/it]
